In [15]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from pydantic import BaseModel, Field
load_dotenv()
import warnings
warnings.filterwarnings(action="ignore")

In [5]:
os.environ["REQUESTS_CA_BUNDLE"] = r"../../ca-bundle-full.crt"

In [6]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
) 

In [10]:
class Playlist(BaseModel):
    name: str = Field(description="Name of the playlist")
    songs: list = Field(description="A list of songs in the playlist")

In [13]:
output_parser = PydanticOutputParser(pydantic_object = Playlist)
format_instructions = output_parser.get_format_instructions()

In [14]:
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the playlist", "title": "Name", "type": "string"}, "songs": {"description": "A list of songs in the playlist", "items": {}, "title": "Songs", "type": "array"}}, "required": ["name", "songs"]}\n```'

In [17]:
human_text = "{instruction}\n{format_instructions}"
message = HumanMessagePromptTemplate.from_template(human_text)
prompt = ChatPromptTemplate.from_messages([message])

chain = prompt | llm | output_parser
chain.invoke({"instruction":"Create a playlist of disco songs","format_instructions":format_instructions})

Playlist(name='Disco Fever', songs=["Stayin' Alive", 'I Will Survive', 'Dancing Queen', 'Le Freak', 'Funky Town', 'Super Freak', 'Boogie Wonderland', 'Y.M.C.A.', "Don't Stop 'Til You Get Enough", 'You Should Be Dancing'])